In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import cv2
import os
import numpy as np
from datetime import datetime, timedelta

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install opencv-python opencv-contrib-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
    --------------------------------------- 0.8/38.8 MB 3.7 MB/s eta 0:00:11
   - -------------------------------------- 1.6/38.8 MB 3.7 MB/s eta 0:00:11
   -- ------------------------------------- 2.1/38.8 MB 3.5 MB/s eta 0:00:11
   -- ------------------------------------- 2.6/38.8 MB 3.0 MB/s eta 0:00:12
   --- ------------------------------------ 3.4/38.8 MB 3.1 MB/s eta 0:00:12
   ---- ----------------------------------- 3.9/38.8 MB 3.2 MB/s eta 0:00:11
   ---- ----------------------------------- 4.5/38.8 MB 3.2 MB/s eta 0:00:11
   ----- ---------------------------------- 5.0/38.8 MB 3.1 MB/s eta 0:00:11
   ----- ---------------------------------- 5.5/38.8 MB 3.0 MB/s eta 0:00:12
   ----- ---------------------------------- 5.8/38.8 MB 2.9 MB/s eta 0:00:12
   ------ --------------------------------- 6.3/38.8 MB 2.7 MB/s eta 0:00:12
   ------ --------------------------------- 6.6/38.8 MB 2.7 MB/s eta 0:00:12
   ---


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
orb = cv2.ORB_create()

# Load facebank
facebank_path = "facebank"
facebank = {}
facebank_descriptors = []

def preprocess_facebank(facebank_path):
    for file in os.listdir(facebank_path):
        filepath = os.path.join(facebank_path, file)
        if file.endswith(('png', 'jpg', 'jpeg')):
            label = os.path.splitext(file)[0]  # Use filename (without extension) as label
            image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
            keypoints, descriptors = orb.detectAndCompute(image, None)
            if descriptors is not None:
                facebank[label] = (keypoints, descriptors)
                facebank_descriptors.append(descriptors)
preprocess_facebank(facebank_path)

In [4]:
index_params = dict(algorithm=6,  # Use the LSH (Locality Sensitive Hashing) index
                    table_number=6,  # Number of hash tables
                    key_size=12,  # Size of the key
                    multi_probe_level=1)  # Number of probes
search_params = dict(checks=50)  # Number of times the trees in the FLANN index will be traversed

flann = cv2.FlannBasedMatcher(index_params, search_params)

def match_face(input_descriptors):
    best_match = None
    best_score = float('inf')
    
    for label, (_, descriptors) in facebank.items():
        # Perform KNN matching (K=2, to get the two best matches)
        matches = flann.knnMatch(input_descriptors, descriptors, k=2)

        # Apply ratio test (Lowe's ratio test) to filter good matches
        good_matches = []
        for match_pair in matches:
            if len(match_pair) == 2:  # Ensure we have two matches to unpack
                m, n = match_pair  # Unpack the two closest matches
                if m.distance < 0.7 * n.distance:  # 0.7 is a commonly used threshold for ratio test
                    good_matches.append(m)

        score = sum([match.distance for match in good_matches]) / len(good_matches) if good_matches else float('inf')

        if score < best_score:
            best_match = label
            best_score = score

    return best_match if best_score < 80 else "Unknown"  # Threshold for matching


In [5]:
# yolo_cfg_url = "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg"
# yolo_weights_url = "https://pjreddie.com/media/files/yolov3.weights"

# # Use OpenCV DNN module to read the model from URL
# yolo_net = cv2.dnn.readNetFromDarknet(yolo_cfg_url, yolo_weights_url)

# # Layer names and output layers
# layer_names = yolo_net.getLayerNames()
# output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]

In [5]:
presence_log = {}

def update_presence_log(name):
    now = datetime.now()
    if name in presence_log:
        presence_log[name]["last_seen"] = now
    else:
        presence_log[name] = {"start_time": now, "last_seen": now}
        
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Process each detected face
    for (x, y, w, h) in faces:
        face = gray_frame[y:y+h, x:x+w]
        keypoints, descriptors = orb.detectAndCompute(face, None)
        
        face_resized = cv2.resize(face, (w * 2, h * 2))
        cv2.imshow("Detected Face", face_resized)
        
        if descriptors is not None:
            name = match_face(descriptors)
            update_presence_log(name)
            
            # Draw rectangle around face and label
            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Display presence log
for person, times in presence_log.items():
    duration = times["last_seen"] - times["start_time"]
    print(f"{person}: Present for {duration}")

Unknown: Present for 0:00:08.565204
ANDREAS: Present for 0:00:08.630670
ENRICO: Present for 0:00:08.734743
Bryan Eugene: Present for 0:00:08.670358


In [6]:
import cv2
import os
from datetime import datetime

# Initialize SIFT
sift = cv2.SIFT_create()

# Load facebank
facebank_path = "./facebank/"
facebank = {}
facebank_descriptors = []

def preprocess_facebank(facebank_path):
    for file in os.listdir(facebank_path):
        filepath = os.path.join(facebank_path, file)
        if file.endswith(('png', 'jpg', 'jpeg')):
            label = os.path.splitext(file)[0]  # Use filename (without extension) as label
            image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
            keypoints, descriptors = sift.detectAndCompute(image, None)
            if descriptors is not None:
                facebank[label] = (keypoints, descriptors)
                facebank_descriptors.append(descriptors)

preprocess_facebank(facebank_path)

# FLANN parameters for SIFT
index_params = dict(algorithm=1,  # KDTree
                    trees=5)  # Number of trees
search_params = dict(checks=50)  # Number of times the trees in the FLANN index will be traversed

flann = cv2.FlannBasedMatcher(index_params, search_params)

def match_face(input_descriptors):
    best_match = None
    best_score = float('inf')
    
    for label, (_, descriptors) in facebank.items():
        # Perform KNN matching (K=2, to get the two best matches)
        matches = flann.knnMatch(input_descriptors, descriptors, k=2)

        # Apply ratio test (Lowe's ratio test) to filter good matches
        good_matches = []
        for match_pair in matches:
            if len(match_pair) == 2:  # Ensure we have two matches to unpack
                m, n = match_pair  # Unpack the two closest matches
                if m.distance < 0.7 * n.distance:  # 0.7 is a commonly used threshold for ratio test
                    good_matches.append(m)

        score = sum([match.distance for match in good_matches]) / len(good_matches) if good_matches else float('inf')

        if score < best_score:
            best_match = label
            best_score = score
        
    
    print(best_score)
    return best_match if best_score < 60 else "Unknown"  # Threshold for matching

presence_log = {}

def update_presence_log(name):
    now = datetime.now()
    if name in presence_log:
        presence_log[name]["last_seen"] = now
    else:
        presence_log[name] = {"start_time": now, "last_seen": now}
        
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Process each detected face
    for (x, y, w, h) in faces:
        face = gray_frame[y:y+h, x:x+w]
        keypoints, descriptors = sift.detectAndCompute(face, None)
        
        face_resized = cv2.resize(face, (w * 2, h * 2))
        cv2.imshow("Detected Face", face_resized)
        
        if descriptors is not None:
            name = match_face(descriptors)
            update_presence_log(name)
            
            # Draw rectangle around face and label
            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Display presence log
for person, times in presence_log.items():
    duration = times["last_seen"] - times["start_time"]
    print(f"{person}: Present for {duration}")


207.84685516357422
208.80224100748697
inf
inf
inf
232.97210693359375
inf
186.8903350830078
180.9800796508789
173.10401916503906
187.8470662434896
190.69641621907553
189.38848876953125
139.25132942199707
187.77535756429037
177.81451416015625
226.13345336914062
210.81679153442383
232.3510284423828
185.49932861328125
153.61965942382812
192.50577545166016
156.9175682067871
163.3183822631836
196.470703125
157.73099772135416
205.88832092285156
183.52932739257812
181.8213348388672
200.57196044921875
214.51972198486328
202.00385284423828
173.0465545654297
132.33668518066406
205.7684555053711
155.73374938964844
178.73724365234375
187.12935892740884
172.76044209798178
195.73271560668945
175.47581990559897
177.302001953125
177.302001953125
181.60778299967447
142.47125244140625
144.72243118286133
137.86468251546225
191.12148666381836
141.7867889404297
190.09004974365234
111.91763687133789
223.04449462890625
203.2976837158203
179.13682556152344
213.83404541015625
202.5290069580078
214.2101745605468

In [8]:
!pip install mediapipe

   ---------------------------------------- 0.0/50.9 MB ? eta -:--:--
    --------------------------------------- 0.8/50.9 MB 8.5 MB/s eta 0:00:06
   - -------------------------------------- 1.6/50.9 MB 4.2 MB/s eta 0:00:12
   - -------------------------------------- 2.4/50.9 MB 3.7 MB/s eta 0:00:14
   -- ------------------------------------- 2.9/50.9 MB 3.5 MB/s eta 0:00:14
   -- ------------------------------------- 3.4/50.9 MB 3.4 MB/s eta 0:00:14
   -- ------------------------------------- 3.7/50.9 MB 3.2 MB/s eta 0:00:15
   --- ------------------------------------ 3.9/50.9 MB 2.7 MB/s eta 0:00:18
   --- ------------------------------------ 4.2/50.9 MB 2.6 MB/s eta 0:00:19
   --- ------------------------------------ 4.7/50.9 MB 2.5 MB/s eta 0:00:19
   --- ------------------------------------ 4.7/50.9 MB 2.5 MB/s eta 0:00:19
   --- ------------------------------------ 5.0/50.9 MB 2.3 MB/s eta 0:00:20
   ---- ----------------------------------- 5.2/50.9 MB 2.1 MB/s eta 0:00:22
   ---


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import os
import numpy as np
from datetime import datetime
import mediapipe as mp

# Initialize ORB
orb = cv2.ORB_create()

# Load facebank
facebank_path = "facebank"
facebank = {}
facebank_descriptors = []

def preprocess_facebank(facebank_path):
    for file in os.listdir(facebank_path):
        filepath = os.path.join(facebank_path, file)
        if file.endswith(('png', 'jpg', 'jpeg')):
            label = os.path.splitext(file)[0]  # Use filename (without extension) as label
            image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
            keypoints, descriptors = orb.detectAndCompute(image, None)
            if descriptors is not None:
                facebank[label] = (keypoints, descriptors)
                facebank_descriptors.append(descriptors)
preprocess_facebank(facebank_path)

# Initialize Matcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

def match_face(input_descriptors):
    best_match = None
    best_score = float('inf')
    
    for label, (_, descriptors) in facebank.items():
        # Match descriptors
        matches = bf.match(input_descriptors, descriptors)
        matches = sorted(matches, key=lambda x: x.distance)
        
        # Calculate score
        score = sum([match.distance for match in matches]) / len(matches) if matches else float('inf')
        
        if score < best_score:
            best_match = label
            best_score = score
    
    return best_match if best_score < 100 else "Unknown"  # Adjust threshold

# Presence log
presence_log = {}

def update_presence_log(name):
    now = datetime.now()
    if name in presence_log:
        presence_log[name]["last_seen"] = now
    else:
        presence_log[name] = {"start_time": now, "last_seen": now}

# Initialize Mediapipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection()

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces using Mediapipe
    results = face_detection.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

            face = gray_frame[y:y+h, x:x+w]
            if face.size == 0:  # Skip empty detections
                continue

            # Resize face for better descriptor quality
            face_resized = cv2.resize(face, (128, 128))
            keypoints, descriptors = orb.detectAndCompute(face_resized, None)

            if descriptors is not None:
                name = match_face(descriptors)
                update_presence_log(name)

                # Draw rectangle and label on frame
                color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Display presence log
for person, times in presence_log.items():
    duration = times["last_seen"] - times["start_time"]
    print(f"{person}: Present for {duration}")


Unknown: Present for 0:00:06.196993
